# LAS Point Cloud Projection to 2D Planes

This notebook demonstrates how to read a LAS file and project it onto different 2D planes at various angles.

## 1. Install and Import Required Libraries

In [ ]:
# Uncomment to install required packages
# !pip install numpy laspy matplotlib

In [ ]:
import numpy as np
import laspy
import matplotlib.pyplot as plt
from pathlib import Path

## 2. Define Helper Functions

In [ ]:
def las2numpy(las):
    """Convert LAS points to numpy array of [x, y, z] coordinates."""
    points = np.vstack([las.x, las.y, las.z]).T
    return points

def find_highest_point(points):
    """Find the point with the maximum z coordinate."""
    max_idx = np.argmax(points[:, 2])
    return points[max_idx]

def offset(points):
    """Calculate offset based on highest point (x, y, 0)."""
    highest = find_highest_point(points)
    return np.array([highest[0], highest[1], 0])

def get_plane(angle_deg):
    """
    Get plane basis vectors for projection at given angle.
    
    Returns:
        u: first basis vector (in xy plane, rotated by angle)
        v: second basis vector (z direction)
        n: normal vector to the plane
    """
    angle = np.radians(angle_deg)
    u = np.array([np.cos(angle), np.sin(angle), 0])
    v = np.array([0., 0., 1.])
    n = np.array([-np.sin(angle), np.cos(angle), 0])
    return u, v, n

def project_to_plane(points, u, v):
    """Project 3D points onto plane defined by basis vectors u and v."""
    projected = np.column_stack([
        np.dot(points, u),
        np.dot(points, v)
    ])
    return projected

def hist2d(points, xbins, ybins):
    """Create 2D histogram of projected points."""
    H, _, _ = np.histogram2d(
        points[:, 0], 
        points[:, 1], 
        bins=[xbins, ybins]
    )
    return H.T  # Transpose to match Julia's indexing

## 3. Load Your LAS File

In [ ]:
# Specify the path to your LAS file
las_file_path = "path/to/your/file.las"  # CHANGE THIS to your actual file path

# Load the LAS file
las = laspy.read(las_file_path)

# Convert to numpy array
points = las2numpy(las)

print(f"Loaded {len(points)} points")
print(f"Point cloud extent:")
print(f"  X: [{points[:, 0].min():.2f}, {points[:, 0].max():.2f}]")
print(f"  Y: [{points[:, 1].min():.2f}, {points[:, 1].max():.2f}]")
print(f"  Z: [{points[:, 2].min():.2f}, {points[:, 2].max():.2f}]")

## 4. Visualize Original 3D Point Cloud

In [ ]:
# Create a 3D scatter plot (subsample for performance if needed)
subsample = 1000  # Show every Nth point
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

sampled_points = points[::max(1, len(points)//subsample)]
ax.scatter(sampled_points[:, 0], 
          sampled_points[:, 1], 
          sampled_points[:, 2], 
          c=sampled_points[:, 2],  # Color by height
          cmap='viridis',
          s=1)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('3D Point Cloud (subsampled)')
plt.tight_layout()
plt.show()

## 5. Shift Points Based on Highest Point

In [ ]:
# Calculate offset
o = offset(points)
print(f"Offset (highest point): X={o[0]:.2f}, Y={o[1]:.2f}, Z={o[2]:.2f}")

# Shift points
shifted_points = points - o

print(f"Shifted point cloud extent:")
print(f"  X: [{shifted_points[:, 0].min():.2f}, {shifted_points[:, 0].max():.2f}]")
print(f"  Y: [{shifted_points[:, 1].min():.2f}, {shifted_points[:, 1].max():.2f}]")
print(f"  Z: [{shifted_points[:, 2].min():.2f}, {shifted_points[:, 2].max():.2f}]")

## 6. Project to a Single Plane (Test with 0 degrees)

In [ ]:
# Test with a single angle first
angle = 0  # degrees

# Get plane basis vectors
u, v, n = get_plane(angle)
print(f"Plane at {angle}°:")
print(f"  u (horizontal): {u}")
print(f"  v (vertical): {v}")
print(f"  n (normal): {n}")

# Project points
projected_points = project_to_plane(shifted_points, u, v)

print(f"\nProjected {len(projected_points)} points")
print(f"Projected extent:")
print(f"  X: [{projected_points[:, 0].min():.2f}, {projected_points[:, 0].max():.2f}]")
print(f"  Y: [{projected_points[:, 1].min():.2f}, {projected_points[:, 1].max():.2f}]")

## 7. Create 2D Histogram

In [ ]:
# Calculate extent and bins
xext = (projected_points[:, 0].min(), projected_points[:, 0].max())
yext = (projected_points[:, 1].min(), projected_points[:, 1].max())

xl = xext[1] - xext[0]
yl = yext[1] - yext[0]

ypx = 1000
xpx = int(ypx / (yl / xl))

# Calculate point density
pd = np.sqrt((xl * yl) / len(projected_points))

print(f"Bin size (point density): {pd:.4f}")
print(f"Image size: {xpx} x {ypx} pixels")

# Create bins
xbins = np.arange(xext[0], xext[1] + pd, pd)
ybins = np.arange(yext[0], yext[1] + pd, pd)

print(f"Number of bins: {len(xbins)} x {len(ybins)}")

# Create 2D histogram
arr = hist2d(projected_points, xbins, ybins)

print(f"Histogram shape: {arr.shape}")
print(f"Point counts range: [{arr.min()}, {arr.max()}]")

## 8. Visualize the Projection

In [ ]:
# Log transform (add 1 to avoid log(0))
larr = np.log(arr + 1)

# Create figure
fig, axes = plt.subplots(1, 2, figsize=(14, 8))

# Raw histogram
im1 = axes[0].imshow(
    arr, 
    cmap='binary',
    aspect='auto',
    origin='lower'
)
axes[0].set_title(f'Raw Histogram (angle={angle}°)')
axes[0].axis('off')
plt.colorbar(im1, ax=axes[0], label='Point count')

# Log-transformed histogram
im2 = axes[1].imshow(
    larr, 
    cmap='binary',
    aspect=xpx / ypx,
    vmin=-1,
    vmax=larr.max() * 0.5,
    origin='lower'
)
axes[1].set_title(f'Log-transformed Histogram (angle={angle}°)')
axes[1].axis('off')
plt.colorbar(im2, ax=axes[1], label='log(count + 1)')

plt.tight_layout()
plt.show()

## 9. Generate All Four Projections

In [ ]:
angles = [0, 45, 90, 135]

fig, axes = plt.subplots(2, 2, figsize=(14, 16))
axes = axes.flatten()

for idx, angle in enumerate(angles):
    # Get plane basis vectors
    u, v, n = get_plane(angle)
    
    # Project points
    projected_points = project_to_plane(shifted_points, u, v)
    
    # Calculate extent and bins
    xext = (projected_points[:, 0].min(), projected_points[:, 0].max())
    yext = (projected_points[:, 1].min(), projected_points[:, 1].max())
    
    xl = xext[1] - xext[0]
    yl = yext[1] - yext[0]
    
    ypx = 1000
    xpx = int(ypx / (yl / xl))
    
    # Calculate point density
    pd = np.sqrt((xl * yl) / len(projected_points))
    
    xbins = np.arange(xext[0], xext[1] + pd, pd)
    ybins = np.arange(yext[0], yext[1] + pd, pd)
    
    # Create 2D histogram
    arr = hist2d(projected_points, xbins, ybins)
    
    # Log transform
    larr = np.log(arr + 1)
    
    # Plot
    im = axes[idx].imshow(
        larr, 
        cmap='binary',
        aspect=xpx / ypx,
        vmin=-1,
        vmax=larr.max() * 0.5,
        origin='lower'
    )
    axes[idx].set_title(f'Projection at {angle}°', fontsize=14)
    axes[idx].axis('off')

plt.suptitle('Tree Projections at Different Angles', fontsize=16, y=0.995)
plt.tight_layout()
plt.show()

## 10. Save Projections to Files

In [ ]:
# Create output directory
output_dir = Path("output_projections")
output_dir.mkdir(exist_ok=True)

filename = Path(las_file_path).stem

for angle in angles:
    # Get plane basis vectors
    u, v, n = get_plane(angle)
    
    # Project points
    projected_points = project_to_plane(shifted_points, u, v)
    
    # Calculate extent and bins
    xext = (projected_points[:, 0].min(), projected_points[:, 0].max())
    yext = (projected_points[:, 1].min(), projected_points[:, 1].max())
    
    xl = xext[1] - xext[0]
    yl = yext[1] - yext[0]
    
    ypx = 1000
    xpx = int(ypx / (yl / xl))
    
    # Calculate point density
    pd = np.sqrt((xl * yl) / len(projected_points))
    
    xbins = np.arange(xext[0], xext[1] + pd, pd)
    ybins = np.arange(yext[0], yext[1] + pd, pd)
    
    # Create 2D histogram
    arr = hist2d(projected_points, xbins, ybins)
    
    # Log transform
    larr = np.log(arr + 1)
    
    # Create and save figure
    fig, ax = plt.subplots(figsize=(6, 8))
    
    im = ax.imshow(
        larr, 
        cmap='binary',
        aspect=xpx / ypx,
        vmin=-1,
        vmax=larr.max() * 0.5,
        origin='lower'
    )
    
    ax.axis('off')
    
    filepath = output_dir / f"{filename}_{angle}.png"
    plt.savefig(filepath, bbox_inches='tight', dpi=100)
    plt.close(fig)
    
    print(f"Saved: {filepath}")

print(f"\nAll projections saved to {output_dir}/")

## 11. Additional Exploration (Optional)

In [ ]:
# Explore different parameters
# Try different colormap
fig, ax = plt.subplots(figsize=(8, 10))

angle = 0
u, v, n = get_plane(angle)
projected_points = project_to_plane(shifted_points, u, v)

xext = (projected_points[:, 0].min(), projected_points[:, 0].max())
yext = (projected_points[:, 1].min(), projected_points[:, 1].max())
xl = xext[1] - xext[0]
yl = yext[1] - yext[0]
ypx = 1000
xpx = int(ypx / (yl / xl))
pd = np.sqrt((xl * yl) / len(projected_points))
xbins = np.arange(xext[0], xext[1] + pd, pd)
ybins = np.arange(yext[0], yext[1] + pd, pd)
arr = hist2d(projected_points, xbins, ybins)
larr = np.log(arr + 1)

# Try different colormaps: 'viridis', 'plasma', 'inferno', 'gray', 'binary_r'
im = ax.imshow(larr, cmap='viridis', aspect=xpx / ypx, origin='lower')
ax.axis('off')
plt.colorbar(im, ax=ax)
plt.title('Alternative Colormap (viridis)')
plt.show()